In [ ]:
print("test")

In [1]:
%pwd

'c:\\Users\\marre\\Desktop\\Kth\\ID1214\\Medical-Chatbot\\research'

In [2]:
import os
os.chdir("../")

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file(data="Data/")

In [8]:
#extracted_data

In [9]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks 

In [10]:
text_chunks=text_split(extracted_data)
print("text length", len(text_chunks))

text length 39994


In [ ]:
text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\marre\AppData\Local\Temp\ipykernel_23800\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [16]:
query_result = embeddings.embed_query("Hello world")
print(len(query_result))

384


In [ ]:
#query_result

In [76]:
from dotenv import load_dotenv
load_dotenv()

True

In [77]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [36]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import os

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "medichatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [78]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print(os.getenv("PCONE_API_KEY"))

pcsk_3j1SCy_52eK8UiRNTbESBXNW7BD473qqGApDW77r9Sv8zg2ZfiztAEvgg2Cm5uZVLzdSwz


In [39]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
    )

In [79]:
# Loading Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [80]:
docsearch

In [81]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [82]:
retrieved_docs = retriever.invoke("What is AIDS?")

In [44]:
retrieved_docs

[Document(id='3c616f8a-832d-4099-b250-ea6ab14f4126', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 3072.0, 'page_label': '3043', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\Medical_book_manual.pdf', 'total_pages': 4505.0}, page_content='involves removing a small piece of prostate tissue for\nmicroscopic examination.\nKEY TERMS\nAIDS— Acquired immunodeficiency syndrome. A\ndisease caused by infection with the human immu-\nnodeficiency virus (HIV). In people with this dis-\nease, the immune system breaks down, opening\nthe door to other infections and some types of\ncancer.\nAntibiotic— A medicine used to treat infections.\nChemotherapy— Treatment of an illness with che-\nmical agents. The term is usually used to describe'),
 Document(id='f405d679-e43e-45e7-8741-12963601e3cd', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22

In [83]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [84]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant and your tasks consist of answering questions."
    "Use the following contexts to answer the question."
    "If you don't know an answer, tell the user that you don't know."
    "You should use 3 sentences maximum and give a concise answer."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [85]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [89]:
response = rag_chain.invoke({"input": "Give me some symptoms for AIDS."})
print(response["answer"])



Some common symptoms of AIDS include low-grade fevers, chronic fatigue, and general weakness. HIV can also cause malabsorption of food, loss of appetite, and increased metabolism, leading to the wasting syndrome. Other symptoms may include yeast infections in the mouth, open sores or ulcers, diarrhea, and other gastrointestinal issues. Late-stage AIDS may also involve diseases of the lungs and kidneys, as well as nerve damage in the arms and legs.
